In [1]:
import sys
import torch
from torch import nn
import os
import numpy as np

sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1

/Users/vik/.virtualenvs/nnets/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from csv_data import WeatherDatasetWrapperRNN

class WeatherDataset(WeatherDatasetWrapperRNN):
    predictors = ["tmax", "tmin", "rain"]
    target = "tmax_tomorrow"
    sequence_length = 7

wrapper = WeatherDataset()
datasets = wrapper.generate_datasets(BATCH_SIZE)

In [3]:
train = datasets["train"]
valid = datasets["validation"]
test = datasets["test"]

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        torch.manual_seed(0)
        self.rnn = nn.RNN(3, 4, 1, batch_first=True)
        self.dense = nn.Linear(4, 1)

    def forward(self, x):
        x, hidden = self.rnn(x)
        x = self.dense(x)
        return x

In [13]:
def mse(actual, predicted):
    return torch.mean((actual-predicted)**2)

In [19]:
model = NeuralNetwork().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

EPOCHS = 10
for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch, (sequence, target) in enumerate(train):
        optimizer.zero_grad()

        sequence = sequence.to(DEVICE)
        pred = model(sequence)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

        epoch_loss += mse(target, pred)

    losses = []
    with torch.no_grad():
        valid_loss = 0
        for batch, (sequence, target) in enumerate(valid):
            sequence = sequence.to(DEVICE)
            pred = model(sequence)
            loss = loss_fn(pred, target)
            valid_loss += mse(target, pred)

    print(f"Epoch {epoch} train loss: {epoch_loss / len(train)} valid loss: {valid_loss / len(valid)}")

Epoch 0 train loss: 321.328857421875 valid loss: 30.11574363708496
Epoch 1 train loss: 25.661727905273438 valid loss: 23.002601623535156
Epoch 2 train loss: 23.3114070892334 valid loss: 22.170818328857422
Epoch 3 train loss: 22.823204040527344 valid loss: 21.30845069885254
Epoch 4 train loss: 22.67506980895996 valid loss: 21.5742130279541
Epoch 5 train loss: 22.448875427246094 valid loss: 21.318944931030273
Epoch 6 train loss: 22.25265121459961 valid loss: 21.07511329650879
Epoch 7 train loss: 22.150007247924805 valid loss: 20.7280216217041
Epoch 8 train loss: 22.084850311279297 valid loss: 21.266441345214844
Epoch 9 train loss: 22.04229164123535 valid loss: 21.165512084960938


In [18]:
[p for p in model.named_parameters()]

[('rnn.weight_ih_l0',
  Parameter containing:
  tensor([[-0.0031,  0.2681, -0.4111],
          [-0.3709, -0.1920,  0.1321],
          [-0.0104,  0.3966, -0.0446],
          [ 0.1317, -0.1510, -0.0986]], requires_grad=True)),
 ('rnn.weight_hh_l0',
  Parameter containing:
  tensor([[-0.4773, -0.3309, -0.2054,  0.0188],
          [ 0.1960,  0.2986, -0.3428, -0.2190],
          [ 0.1814,  0.4151, -0.1033,  0.3740],
          [-0.0809,  0.0528,  0.4521, -0.4641]], requires_grad=True)),
 ('rnn.bias_ih_l0',
  Parameter containing:
  tensor([-0.3163, -0.1189, -0.1941,  0.4332], requires_grad=True)),
 ('rnn.bias_hh_l0',
  Parameter containing:
  tensor([-0.3256, -0.2224, -0.3485, -0.4670], requires_grad=True)),
 ('dense.weight',
  Parameter containing:
  tensor([[-0.2950,  0.4278,  0.2163,  0.2398]], requires_grad=True)),
 ('dense.bias',
  Parameter containing:
  tensor([0.0388], requires_grad=True))]